In [1]:
from math import *
from vpython import *
import pandas as pd

<IPython.core.display.Javascript object>

In [2]:
#constants
NA = 6.023e23
g = 9.8 #assume constant
r = 8.314
kB = 1.380649e-23
P0 = 101325 #N/m**2, pressure at sea level
Ti = 293 #Kelvin, temperature at sea level
h = 10 #heat transfer coefficient for open convection
airMoleculeMass = 24.81e-26 #kg

#functions

#environmental conditions
#Ti: the temperature at sea level, h: the height of the balloon
def environmentTemp(height):
    return Ti - h/1000 * 9.8
#P0: sea level pressure, height: the height of the balloon, temperature: the temperature of the air
def atmoPressure(height, temperature): #P0 = sea level pressure
    return P0 * pow(e, (-airMoleculeMass*g*height/(kB*temperature)))
#pressure: the air pressure, temperature: the temperature of the air
def atmoDensity(pressure, temperature):
    return pressure/((kB/airMoleculeMass)*temperature)

#use ideal gas law to calculate n (number of particles in balloon)
def nIdealGasLaw(volume, pressure, temperature):
    return pressure*volume/(r*temperature)*NA
def massDensityAir(numAirParticles, volume):
    return numAirParticles*airMoleculeMass/volume 
#h: heat transfer coefficient, A: heat transfer surface area, 
#Tt:time-dependent temperature of the object's surface, Te: temperature of the environment
def Newton(h, A, Tt, Te): #Newton's law of cooling, output is dQ/dt
    return h * A * (Tt-Te)

#Force equations
def buoyancy(outsideDensity,balloon):
    buoyancy = vector(0, 0, outsideDensity * g * balloon.volume)
    return buoyancy
def gravity(mass):
    gravity = vector(0, 0, mass * (-g))
    return gravity
def airResistance(outsideDensity,balloon):
    dragCoefficient = 0.47
    crossArea = pi * (balloon.radius)**2
    airResistance = 0.5 * outsideDensity * (mag(balloon.velocity)**2) * dragCoefficient * crossArea*(-hat(balloon.velocity))
    return airResistance

# Energy equations
def KE(mass, momentum):
    return 1/2 * mag(momentum)**2 / mass
def Ug(mass, height):
    return mass*g * height

# Heat/temperature conversion
#t0: the temperature in last loop, mass: the mass of the object, 
#heat: heat changed from last loop, c:specific heat
def heatToTemp(t0, mass, heat, c): #t0: the temperature in last loop, c:specific heat
    return t0 + heat/(mass*c)

heightPlot = graph(title='Hot air balloon endurance',xtitle='fuel(kg)',ytitle='maximum height reached (m)',xmin=0,ymin=0)
drawHeight = gcurve(color=color.black,label='height')
hdf = pd.DataFrame(columns = ['fuel', 'height'])

for i in (range(75, 101, 5)):
    scene1 = canvas(title="Hot air balloon",caption="caption here",center=vector(0,0,0),
             width=1000, height=1000,background=color.blue)
    #create graphs here

#     Fplot = graph(title='Force plot',xtitle='time (s)',ytitle='force (N)')
#     drawFg = gcurve(color=color.purple,label='Gravity')
#     drawFb = gcurve(color=color.blue,label='Buoyancy')
#     drawFa = gcurve(color=color.cyan,label='Air resistance')
#     drawQ = gcurve(color=color.red, label='Heat in balloon (J)')
#     drawH = gcurve(color=color.black, label='Height (m)')

#     Eplot=graph(title='Energy plot',xtitle='height(m)',ytitle='Energy(J)',xmin=0,ymin=0)
#     drawKE = gcurve(color=color.green,label='KE/10^4')
#     drawUg = gcurve(color=color.blue,label='Ug/10^8')
#     drawQ = gcurve(color=color.red,label='Q/10^9')
#     drawUc = gcurve(color=color.orange, label='Uchem/10^9')
#     drawEtot = gcurve(color=color.black, label='Total/10^9')
    
    #create objects
    ground = box(pos = vector(0, 0, 0), size=vector(50, 50, 0), color=color.green)
    basket = box(pos = vector(0, 0, 0.5), size =vector(1, 1, 1), color=color.yellow)
    balloon = sphere(pos = vector(0, 0, 10.4), radius = 8.4, color=color.yellow)
    fire = sphere(pos = vector(0, 0, 3), radius = 1, color=color.red)

    #set initial conditions(velocity, etc) here
    AtmoTemp=293 #room temperature
    BallonTemp=293 #room temperature

    balloon.velocity = vector(0,0,0)
    balloon.momentum=vector(0, 0, 0)
    balloon.volume=4/3*pi*balloon.radius**3
    balloon.shellMass = 100
    balloon.basketMass = 300
    balloon.surfaceArea = 4*pi*balloon.radius**2
    balloon.t0 = Ti
    balloon.temp = Ti
    balloon.q = 0
    balloon.c = 1.67 * 1000 #j/kg 
    internalDensity = massDensityAir(nIdealGasLaw(balloon.volume, atmoPressure(balloon.pos.z, Ti), Ti), balloon.volume)

    fire.q = 0
    fire.temp = Ti
    fire.t0 = Ti
    fire.surfaceArea = 4*pi*fire.radius**2


    #set fuel amount, fuel burn rate here
    fuelmass = i
    burnRate = 0.0157725*.493 #L/s * kg/L = kg/s
    dqdtFire = burnRate*50e6 #50e6 is joules/kg 
    efficiency = 0.7 #the efficiency for the fuel convert into energy

    deltaQ = 0 #total heat transfer from balloon to outside


    t = 0
    dt = 0.5
    runRate = 1000

    while (balloon.momentum.z>=0 and (fuelmass>0 or balloon.momentum.z>0)):
        rate(runRate)
        t += dt

        #calculate buoyancy
        externalTemperature = environmentTemp(height = balloon.pos.z)
        externalPressure = atmoPressure(height = balloon.pos.z, temperature = externalTemperature)
        externalDensity = atmoDensity(pressure = externalPressure, temperature = externalTemperature)
        Fb = buoyancy(outsideDensity = externalDensity, balloon=balloon)

        #figure out how much thermal energy is transferred to the system
        if fuelmass>0:
            dqdtFireBalloon = dqdtFire * efficiency
            fuelmass -= burnRate*dt
        else:
            dqdtFireBalloon = 0
            fire.color=color.black


        #Newton's law of cooling for baloon-->atmosphere
        dqdtBalloonAtmo = Newton(h = h, A = balloon.surfaceArea, Tt = balloon.temp, Te = externalTemperature)

        # update heat in balloon, convert to temperature
        balloon.q += (dqdtFireBalloon - dqdtBalloonAtmo)*dt
        balloon.temp = heatToTemp(t0=balloon.t0,mass=(balloon.shellMass+internalDensity*balloon.volume),heat=(dqdtFireBalloon-dqdtBalloonAtmo)*dt,c=balloon.c)

        #calculate gravity
        internalParticles = nIdealGasLaw(volume=balloon.volume, pressure=externalPressure,temperature=balloon.temp)
        internalDensity = massDensityAir(numAirParticles=internalParticles,volume=balloon.volume)
        balloon.totalMass = balloon.shellMass+internalDensity*balloon.volume+balloon.basketMass
        Fg = gravity(mass=balloon.totalMass)

        #calculate air resistance
        Fa = airResistance(externalDensity, balloon)

        #add gravity, buoyancy, air resistance to get total force
        Ftotal = Fg+Fb+Fa
        if basket.pos.z <=0.5 and Ftotal.z<0:
            Ftotal = vector(0, 0, 0)

        #update momentum
        balloon.momentum += Ftotal * dt
        balloon.velocity = balloon.momentum/balloon.totalMass

        #update position
        balloon.pos += balloon.velocity*dt
        basket.pos += balloon.velocity*dt
        fire.pos += balloon.velocity*dt

        balloon.t0 = balloon.temp
        scene1.camera.follow(balloon)
        if (Ftotal.z>0):
            balloon.opacity = 0.5
#         drawFg.plot(t, Fg.z/1e4)
#         drawFb.plot(t, Fb.z/1e4)
#         drawFa.plot(t, Fa.z)
#         drawQ.plot(t, balloon.q/1e6)
#         drawH.plot(t, balloon.pos.z)

#         drawKE.plot(balloon.pos.z, KE(mass=balloon.totalMass,momentum=balloon.momentum)/1e4)
#         drawUg.plot(balloon.pos.z, Ug(mass=balloon.totalMass,height=balloon.pos.z)/1e8)
#         drawQ.plot(balloon.pos.z, balloon.q/1e9)
#         drawUc.plot(balloon.pos.z, fuelmass*50e6/1e9)
#         drawEtot.plot(balloon.pos.z, (KE(mass=balloon.totalMass,momentum=balloon.momentum)+Ug(mass=balloon.totalMass,height=balloon.pos.z)+balloon.q+fuelmass*50e6)/1e9)

    drawHeight.plot(i,balloon.pos.z)
    hdf.loc[len(hdf.index)] = [i,balloon.pos.z] 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [3]:
hdf

,fuel,height
0,75.0,2154.674989
1,80.0,2154.693682
2,85.0,2154.701514
3,90.0,2154.704794
4,95.0,2154.706169
5,100.0,2154.706744
